# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [8]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/05/18/2025-ai-executive-briefing/',
 'https://edwarddonner.com/2025/04/21/the-complete-agentic-ai-engineering-course/',
 'https://edwarddonner.com/2025/04/21/the-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [9]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [10]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [11]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [12]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [12]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddonner.com/2025/05/28/connecting-my-courses-become-an-llm-expert-and-leader/
https://edwarddo

In [13]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [14]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/black-forest-labs/FLUX.1-Kontext-dev',
 '/google/gemma-3n-E4B-it',
 '/THUDM/GLM-4.1V-9B-Thinking',
 '/tencent/Hunyuan-A13B-Instruct',
 '/kyutai/tts-1.6b-en_fr',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/black-forest-labs/FLUX.1-Kontext-Dev',
 '/spaces/ilcve21/Sparc3D',
 '/spaces/AIDC-AI/Ovis-U1-3B',
 '/spaces/tencent/Hunyuan3D-2.1',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/HuggingFaceFW/fineweb-2',
 '/datasets/facebook/seamless-interaction',
 '/datasets/FreedomIntelligence/ShareGPT-4o-Image',
 '/datasets/black-forest-labs/kontext-bench',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Wri

In [15]:
get_links("https://huggingface.co")

{'links': [{'type': 'about page', 'url': 'https://huggingface.co'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'company page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [16]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [34]:
#print(get_all_details("https://huggingface.co"))

In [18]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [19]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [20]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'docs page', 'url': 'https://huggingface.co/docs'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nCommunity\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nExplore AI Apps\nor\nBrowse 1M+ models\nTrending on\nthis week\nModels\nblack-forest-labs/FLUX.1-Kontext-dev\nUpdated\n9 days ago\n•\n163k\n•\n1.38k\ngoogle/gemma-3n-E4B-it\nUpdated\n4 days ago\n•\n212k\n•\n483\nTHUDM/GLM-4.1V-9B-Thinking\nUpdated\n4 days ago\n•\n8.58k\n•\n227\ntencent/Hunyuan-A13B-Instruct\nUpdated\n5 days ago\n•\n20.8k\n•\n732\nkyutai/tts-1.6b-en_fr\nUpdated\n4 days ago\n•\n9.29k\n•\n203\nBrowse 1M+ models\nSpaces\nRunning\n9.35k\n9.35k\nDee

In [27]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [22]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'company profile', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

## About Us
**Hugging Face** is the forefront of the machine learning community dedicated to building innovative AI solutions. Our platform fosters collaboration, enabling users to create, share, and discover over 1 million models, hundreds of thousands of datasets, and countless applications. We are committed to driving the future of AI by harnessing the power of open-source technology and community contributions.

## Our Offerings
- **Models**: Access and deploy a vast library of ML models across various modalities including text, image, audio, and 3D.
- **Datasets**: Browse and share thousands of datasets tailored for any machine learning task.
- **Spaces**: Run applications effortlessly with a collaborative infrastructure designed for AI projects.

## Community & Collaboration
We pride ourselves on being a community-driven platform. Over 50,000 organizations, including tech giants like Google, Amazon, and Microsoft, actively use our resources. By bringing together experts and novices alike, we constantly push the boundaries of what’s possible in machine learning.

## Company Culture
At Hugging Face, we foster a culture of inclusivity, teamwork, and continuous learning. Our employees are encouraged to collaborate and innovate in a supportive environment, helping them to grow both professionally and personally. 

## Careers
Join a vibrant community of AI enthusiasts! We’re always looking for talented individuals who are passionate about machine learning and open-source solutions. Check our [careers page](#) for current job openings and be part of our mission to build the future of AI together.

## Get Involved
Ready to dive into the world of AI? 
- **Explore AI Apps**: Start creating your own applications using our advanced tools.
- **Join our Community**: Share your work, collaborate on projects, and contribute to discussions in the Hugging Face community.
- **Get Started**: Sign up now to unlock the full potential of our AI tools and resources.

## Connect with Us
Stay updated with our latest advancements and join the conversation on social media. Find us on [GitHub](#), [Twitter](#), [LinkedIn](#), and [Discord](#).

---

Embrace the future of AI with Hugging Face – A community built on collaboration, innovation, and open-source values!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [31]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short and hilarious brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

In [32]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "developer", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [24]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Brochure

---

## **Company Overview**

### **Welcome to Hugging Face!**
Hugging Face is at the forefront of the artificial intelligence community, dedicated to building innovative tools and models that shape the future of AI. Our platform fosters collaboration among machine learning practitioners, allowing users to create, discover, and share an extensive array of models and datasets.

### **What We Offer**
- **1 Million+ Models:** Explore cutting-edge machine learning models to bolster your projects.
- **250k+ Datasets:** Access a rich collection of datasets suitable for diverse ML tasks.
- **Unique Spaces:** Collaborate on applications that integrate machine learning in creative ways.

---

## **Company Culture**
At Hugging Face, we believe in the power of collaboration and community. We nurture a diverse, inclusive environment where team members innovate and experiment together. Our open-source ethos encourages contributions from individuals and organizations alike, making us a vibrant hub for knowledge sharing and skill development in the realm of machine learning.

## **Our Customers**
We are proud to serve over **50,000 organizations**, including industry leaders such as:
- **Meta**
- **Amazon**
- **Google**
- **Microsoft**
  
Our platform supports both non-profits and enterprises, ensuring that everyone can benefit from advanced AI tools.

---

## **Careers at Hugging Face**
Join us on our mission as we expand the possibilities of machine learning! We are looking for innovative minds who are passionate about AI. Whether you're an engineer, researcher, or designer, you'll find ample opportunity to grow your career while contributing to meaningful projects.

### **Why Work with Us?**
- **Collaborative Environment:** Work alongside some of the brightest minds in AI.
- **Diversity & Inclusion:** We are committed to building an inclusive community where everyone feels welcome.
- **Professional Development:** We encourage continual learning through mentorship and resources.

---

## **Get Involved**
Ready to start your journey with us? Whether you want to explore our platforms, collaborate with the community, or join our team, there’s a place for you at Hugging Face!

- **Explore AI Apps:** [Browse 1M+ models](https://huggingface.co)
- **Join our Community:** Participate in discussions and contribute to our open-source projects.
- **Careers:** Check out our [job opportunities](https://huggingface.co/jobs) and become part of the future of AI.

---

## **Connect with Us**
For more information, follow us on our social platforms:
- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://linkedin.com/company/huggingface)

---

*Join Hugging Face and be a part of the AI revolution! Together, we can build tools that empower a smarter tomorrow.*

In [33]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}]}


# Hugging Face: Tickle Your AI Fancy 🤗

Welcome to Hugging Face, the cuddliest corner of the AI universe! This is where machine learning models huddle together, datasets mingle, and innovators from all walks of life turn ideas into reality while sipping lattes and possibly hugging it out.

## Who Are We? 🥳

We're the AI community building the future, one model at a time! With over **1M+ models** and **250k+ datasets**, we're the flourishing garden of techy goodness you didn’t know you were missing. Here at Hugging Face, we strive to make artificial intelligence as friendly as a puppy and as collaborative as a group project gone right!

## What’s in the Hug? 🏆

- **Models Galore**: From cute chatbots to mind-boggling 3D generators, we're home to some sassy models that are making waves in the tech world. Check out our trending models like **THUDM/GLM-4.1V-9B-Thinking** (big brains welcome).

- **Spunky Spaces**: Explore environments running AI applications that help you design, create, and doodle while you work. Complete with emojis! 🐳⚡🎨👻

- **Datasets for Days**: Adventure through our datasets so abundant it may feel like an AI Elysium! Dive into **awesome-chatgpt-prompts** or get your groove on with **fineweb-2**.

- **Community Spirit**: Join over **50,000 organizations** including titans like Amazon, Google, and even your grandma's knitting club (just kidding, but you get the point)! 🤷‍♀️

## Company Culture: The Warm Cuddle Zone 💖

At Hugging Face, we embrace collaboration like it’s a group hug! Here, your voice matters, failure is just a learning step, and creativity is encouraged – preferably while wearing fuzzy socks. Join our passionate community, meet quirky professionals, and engage in crystal-clear dialogues (unlike those gloomy office meetings).

## Career Opportunities: Work with Pakka-Passion🎉

Looking for a job that doesn't feel like a job? Well, we can't promise free puppies, but at Hugging Face, you’ll find an exciting array of career opportunities that’ll make you say “OMG, I want to do that!” From machine learning magicians to data wranglers, check out our openings where you can spin tales of technology and creativity. 

## Why Hugging Face? 🤔

- **Speedy AI**: Navigate the world of AI faster than you can say “TensorFlow”!
- **Open Source²**: We're the geniuses behind popular open-source projects like Transformers and Tokenizers (yes, we have awesome parties).
- **Enterprise-ready**: We take your company to infinity and beyond (with enterprise-grade security to boot)!

## Let's Connect! 🔌

Are you ready to join the AI revolution and sprinkle a little hugging magic in your life? Come explore, **[Sign Up](#)**, and see what the excitement is all about! Don’t forget to follow us on **GitHub, Twitter, LinkedIn,** and every other platform where we show off our cool features!

---

Join us and let’s create a brighter, friendlier AI together because at Hugging Face, we believe in the power of community and some cuddly tech! 🤗✨

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>